# 39.How would you create a basic Flask route that displays "Hello, World!" on the homepage?

In [ ]:
pip install Flask

In [ ]:
from flask import Flask

# Create an instance of the Flask class
app = Flask(__name__)

# Define the route for the homepage
@app.route('/')
def hello_world():
    return 'Hello, World!'

# Run the application
if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


# 40.Explain how to set up a Flask application to handle form submissions using POST requests.

In [ ]:
from flask import Flask, render_template, request

# Initialize the Flask application
app = Flask(__name__)

# Route to display the form and handle the form submission
@app.route('/', methods=['GET', 'POST'])
def form_handler():
    if request.method == 'POST':
        # Retrieve form data
        name = request.form['name']
        email = request.form['email']

        # Process the data (for now, just return it)
        return f"Name: {name}, Email: {email}"

    # Render the form on GET request
    return render_template('form.html')

# Run the application
if __name__ == '__main__':
    app.run(debug=True)


# 41.Write a Flask route that accepts a parameter in the URL and displays it on the page.

In [ ]:
from flask import Flask

app = Flask(__name__)

@app.route('/display/<parameter>')
def display_parameter(parameter):
    return f"You entered: {parameter}"

if __name__ == '__main__':
    app.run(debug=True)


# 42.How can you implement user authentication in a Flask application?

In [ ]:
from flask import Flask, render_template, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
db = SQLAlchemy(app)
login_manager = LoginManager(app)
login_manager.login_view = 'login'

# User model
class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)

# Load user function
@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

# Route for registration
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')
        hashed_password = generate_password_hash(password, method='sha256')

        new_user = User(username=username, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()

        flash('Registration successful! Please log in.')
        return redirect(url_for('login'))

    return render_template('register.html')

# Route for login
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form.get('username')
        password = request.form.get('password')

        user = User.query.filter_by(username=username).first()
        if user and check_password_hash(user.password, password):
            login_user(user)
            return redirect(url_for('dashboard'))
        else:
            flash('Login failed. Please check your username and password.')

    return render_template('login.html')

# Protected dashboard route
@app.route('/dashboard')
@login_required
def dashboard():
    return f"Hello, {current_user.username}! Welcome to your dashboard."

# Logout route
@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

if __name__ == '__main__':
    db.create_all()
    app.run(debug=True)


# 43.Describe the process of connecting a Flask app to a SQLite database using SQLAlchemy.

In [ ]:
pip install Flask SQLAlchemy

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)

# Configure the SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///your_database_name.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Initialize the SQLAlchemy instance
db = SQLAlchemy(app)


In [ ]:
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    email = db.Column(db.String(120), unique=True, nullable=False)

    def __repr__(self):
        return f'<User {self.username}>'


# 44.How would you create a RESTful API endpoint in Flask that returns JSON data?

In [ ]:
from flask import Flask, jsonify

app = Flask(__name__)

In [ ]:
@app.route('/api/data', methods=['GET'])
def get_data():
    data = {
        "name": "Sourav",
        "age": 25,
        "profession": "Data Scientist"
    }
    return jsonify(data)


In [ ]:
@app.route('/api/users', methods=['GET'])
def get_users():
    users = [
        {"id": 1, "name": "Sourav", "profession": "Data Scientist"},
        {"id": 2, "name": "Rahul", "profession": "Software Engineer"},
        {"id": 3, "name": "Priya", "profession": "Product Manager"}
    ]
    return jsonify(users)


# 45.Explain how to use Flask-WTF to create and validate forms in a Flask application.

In [ ]:
pip install Flask-WTF

In [ ]:
from flask import Flask, render_template, redirect, url_for, flash
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import DataRequired, Email, Length

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Required for CSRF protection


In [ ]:
class LoginForm(FlaskForm):
    username = StringField('Username', validators=[DataRequired(), Length(min=4, max=20)])
    email = StringField('Email', validators=[DataRequired(), Email()])
    password = PasswordField('Password', validators=[DataRequired(), Length(min=6)])
    submit = SubmitField('Login')


In [ ]:
@app.route('/login', methods=['GET', 'POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        # Process the form data here
        username = form.username.data
        email = form.email.data
        password = form.password.data
        flash('Form submitted successfully!', 'success')
        return redirect(url_for('login'))

    return render_template('login.html', form=form)


# 46.How can you implement file uploads in a Flask application?

In [ ]:
from flask import Flask, request, redirect, url_for, render_template, flash
import os

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'
app.config['UPLOAD_FOLDER'] = 'uploads'  # Directory to save uploaded files
app.config['ALLOWED_EXTENSIONS'] = {'png', 'jpg', 'jpeg', 'gif'}  # Allowed file extensions

if not os.path.exists(app.config['UPLOAD_FOLDER']):
    os.makedirs(app.config['UPLOAD_FOLDER'])


In [ ]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in app.config['ALLOWED_EXTENSIONS']


In [ ]:
@app.route('/upload', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            flash('File successfully uploaded')
            return redirect(url_for('upload_file'))
    return render_template('upload.html')


# 47.Describe the steps to create a Flask blueprint and why you might use one.

In [ ]:
from flask import Flask

app = Flask(__name__)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
from flask import Blueprint, render_template

# Define the blueprint
users_bp = Blueprint('users', __name__)

# Define routes for the blueprint
@users_bp.route('/profile')
def profile():
    return render_template('profile.html')

@users_bp.route('/settings')
def settings():
    return render_template('settings.html')


In [ ]:
from flask import Blueprint, render_template

# Define the blueprint
users_bp = Blueprint('users', __name__)

# Define routes for the blueprint
@users_bp.route('/profile')
def profile():
    return render_template('profile.html')

@users_bp.route('/settings')
def settings():
    return render_template('settings.html')


# 48.How would you deploy a Flask application to a production server using Gunicorn and Nginx?

In [ ]:
[Unit]
Description=Gunicorn instance to serve myapp
After=network.target

[Service]
User=your_username
Group=www-data
WorkingDirectory=/path/to/your/app
Environment="PATH=/path/to/your/app/venv/bin"
ExecStart=/path/to/your/app/venv/bin/gunicorn -w 4 -b 0.0.0.0:8000 app:app

[Install]
WantedBy=multi-user.target


# 49. Make a fully functional web application using flask, Mangodb. Signup,Signin page.And after successfully login .Say hello Geeks message at webpage.





In [ ]:
from flask import Flask, render_template, request, redirect, url_for, flash, session
from flask_pymongo import PyMongo
from werkzeug.security import generate_password_hash, check_password_hash

app = Flask(__name__)
app.config['SECRET_KEY'] = 'your_secret_key'  # Change this to a random secret key
app.config['MONGO_URI'] = 'mongodb://localhost:27017/mydatabase'  # MongoDB URI

mongo = PyMongo(app)

@app.route('/')
def home():
    if 'username' in session:
        return render_template('home.html', username=session['username'])
    return redirect(url_for('signin'))

@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        hashed_password = generate_password_hash(password, method='sha256')

        users = mongo.db.users
        if users.find_one({'username': username}):
            flash('Username already exists')
            return redirect(url_for('signup'))

        users.insert_one({'username': username, 'password': hashed_password})
        flash('Signup successful! Please sign in.')
        return redirect(url_for('signin'))

    return render_template('signup.html')

@app.route('/signin', methods=['GET', 'POST'])
def signin():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']

        users = mongo.db.users
        user = users.find_one({'username': username})

        if user and check_password_hash(user['password'], password):
            session['username'] = username
            return redirect(url_for('home'))
        else:
            flash('Invalid username or password')

    return render_template('signin.html')

@app.route('/logout')
def logout():
    session.pop('username', None)
    return redirect(url_for('signin'))

if __name__ == '__main__':
    app.run(debug=True)
